<div  style="min-height: 10px; background-color: lightblue; padding: 20px; border: 2px solid blue; border-radius: 10px; width: 900px; margin: 20px auto;">
    
  <center><strong> Capstone Project: AI Calendar Assistant (Multi-Agent GenAI)  </strong></center>

</div>


# Capstone Project: AI Calendar Assistant (Multi-Agent GenAI)

This notebook demonstrates a production-style Generative AI assistant that helps manage Google Calendar events using natural language. It leverages **multi-agent reasoning**, **structured outputs**, **function calling**, and **real-world integration** with the **Google Calendar API**.

## Use Case

Imagine you're a busy professional. Instead of manually creating, deleting, or checking calendar events, this assistant can do it for you via natural conversation like:

> "List all my meetings for today"  
> "Schedule a sync with the team on Sunday at 9 AM"  
> "Delete the event 'Demo Call'"

This is not just a chatbot — it's a GenAI-powered multi-agent system that understands your request, plans a response, and executes calendar actions using structured function calls.

---



## Architecture Overview

### Pipeline Flow

```mermaid
graph TD
    A[User Input] --> B[Intent Detector]
    B -- Not Calendar Related --> C[Small Talk Agent]
    B -- Calendar Related --> D[Planner Agent - Gemini]
    D --> E{Action Type?}
    E -- create_event --> F[Scheduler Agent]
    E -- list_events --> G[List Events Tool]
    E -- delete_event --> H[Delete Tool]
    E -- update_event --> I[Coming Soon]
    E -- unknown --> J[Fallback Response]
```

- **Planner LLM**: Interprets user request and emits structured JSON (with action_type, summary, time, etc.)
- **Scheduler Agent**: Takes JSON and schedules events using `create_event()`
- **Tool Layer**: LangChain tools handle actual calendar operations
- **Calendar API**: Real-time interaction with the user's Google Calendar

---



##  Tech Stack & GenAI Features Used

| Component                          | Purpose                                           |
|-----------------------------------|---------------------------------------------------|
| **Gemini 2.0 Flash**              | LLM to generate structured outputs (JSON)         |
| **LangChain Agents + Tools**      | Multi-agent orchestration and tool calling        |
| **Google Calendar API**           | Real-time scheduling, deletion, and querying      |
| **Structured Output**             | JSON mode to control model output                 |
| **Function Calling**              | Execute external tools via model-generated plans  |
| **Retry Logic**                   | Handles rate-limiting gracefully                  |
| **Kaggle Secrets / Auth**         | Secure API key handling for Google integrations   |

---



##  GenAI Output Evaluation

To ensure reliable function calling and agent routing, we evaluate the planner agent's output for validity.

###  Evaluation Strategy
We check if the planner produces valid JSON with required keys like `"action_type"`. This is crucial because invalid planner output can break the entire agent chain.

Below is a basic schema validator used to test planner outputs:


In [40]:

import json

# Basic structured output validation
def validate_planner_json(json_text: str) -> bool:
    try:
        data = json.loads(json_text)
        return isinstance(data, dict) and "action_type" in data
    except json.JSONDecodeError:
        return False

# Example test
sample_output = '{"action_type": "list_events"}'
print(" Is valid planner output?", validate_planner_json(sample_output))


 Is valid planner output? True



##  Prompt Engineering Strategy

We use prompt engineering to ensure the Gemini LLM produces consistently structured JSON output.

Key strategies:
- Used **few-shot examples** to guide LLM behavior
- Instructed LLM to **only return JSON**, not Markdown or prose
- Injected **dynamic dates** (today/tomorrow) using Python
- Escaped curly braces using double brackets to prevent prompt format errors

This ensures the `planner` always produces actionable structured outputs like:

```json
{"action_type": "create_event", "summary": "Team Sync", "start_time": "..."}
```

These outputs are then interpreted by the scheduler agent or directly routed to tool functions.



## Example Scenarios

Try interacting with the agent using real instructions:

- `"What events do I have today?"` → Planner classifies as `list_events`
- `"Schedule AI sync at 5pm tomorrow"` → Planner emits JSON, Scheduler executes
- `"Cancel meeting 'Sprint Review'"` → Planner sends `delete_event`, Tool finds and deletes

The planner → scheduler setup mimics how a human assistant would operate — planning before executing.

---



##  Final Thoughts

This assistant is a great example of how multi-agent GenAI systems can go beyond chat to deliver task automation in real-world workflows.

Handles real calendar data  
Multi-agent pipeline with clean separation  
Uses LLMs for planning and tool invocation  
Prompt engineered for structure and control  
Robust fallback logic for errors

Future plans to enhance this with features like:
- Event rescheduling
- Multi-user support
- Embedding-based memory (RAG)

--- 


## Setting up the kaggle environment and installing the necessary packages

In [41]:
# 🧹 Remove conflicting packages from the Kaggle environment
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai

# Install required libraries 
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

# Install Google Generative AI client
!pip install -U -q "google-genai==1.7.0"

# Install Google Calendar API and environment helpers
!pip install -qU google-auth google-auth-oauthlib google-api-python-client python-dotenv


## Imports

In [42]:
from google import genai
from google.genai import types
from IPython.display import HTML, Markdown, display

In [43]:

import os
import json
import datetime
from typing import Optional
import re


from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI


## Code for working around the per-minute quota of gemini

In [44]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

## Using credentials to login in Gemini API and Calendar API

###  How to Get Your Gemini API Key (Vertex AI)

1. Go to the [Google AI Studio](https://makersuite.google.com/app) or [Vertex AI Console](https://console.cloud.google.com/vertex-ai).
2. Make sure billing is enabled for your project.
3. In the **Navigation Menu**, go to `Vertex AI` → `API Keys`
4. Click **Create API Key**
5. Copy the generated API key
6. In your **Kaggle Notebook**:
    - Go to `Add-ons` → `Secrets`
    - Add a new secret with:
        - **Key**: `GOOGLE_API_KEY`
        - **Value**: *(paste your API key here)*

**In this notebook, the vertex AI key is copied into a kaggle secret key **GOOGLE_API_KEY**


###  How to Generate `credentials.json` for Google Calendar API

1. Go to the [Google Cloud Console](https://console.cloud.google.com/).
2. Create a new project (or select an existing one).
3. Enable the **Google Calendar API**:
    - Navigate to `APIs & Services` → `Library`
    - Search for **Google Calendar API** and click **Enable**
4. Go to `APIs & Services` → `Credentials`:
    - Click **Create Credentials** → **OAuth Client ID**
    - If prompted, click **Configure Consent Screen**
        - Choose **External**
        - Fill in the app name, support email, etc.
        - Save and continue with defaults
    - Choose **Desktop App** as the application type
    - Click **Create**, then **Download** the `credentials.json` file
5. Upload the `credentials.json` file to your notebook environment 

** In this notebook, the contents of credentials.json file is copied into a kaggle secret key **GOOGLE_CREDENTIALS_JSON**

In [45]:
import os
from kaggle_secrets import UserSecretsClient

# Load secrets from Kaggle

secrets_client = UserSecretsClient()
GOOGLE_API_KEY = secrets_client.get_secret("GOOGLE_API_KEY")
GOOGLE_CREDENTIALS_JSON = secrets_client.get_secret("GOOGLE_CREDENTIALS_JSON")

#Set the Gemini API key as environment variable
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Write credentials.json from secret
with open("credentials.json", "w") as f:
    f.write(GOOGLE_CREDENTIALS_JSON)

In [46]:
import os
from kaggle_secrets import UserSecretsClient

# Load secrets from Kaggle

secrets_client = UserSecretsClient()
GOOGLE_API_KEY = secrets_client.get_secret("GOOGLE_API_KEY")
GOOGLE_CREDENTIALS_JSON = secrets_client.get_secret("GOOGLE_CREDENTIALS_JSON")

#Set the Gemini API key as environment variable
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Write credentials.json from secret
with open("credentials.json", "w") as f:
    f.write(GOOGLE_CREDENTIALS_JSON)

In [47]:
from google_auth_oauthlib.flow import Flow
from google.auth.transport.requests import Request
import pickle
import os

SCOPES = ['https://www.googleapis.com/auth/calendar']

def authenticate_google():
    creds = None

    # Load previously saved token
    if os.path.exists('token.pkl'):
        with open('token.pkl', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = Flow.from_client_secrets_file(
                'credentials.json',
                scopes=SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'
            )

            auth_url, _ = flow.authorization_url(prompt='consent')
            print("Go to this URL and authorize access:\n", auth_url)

            code = input("Paste the authorization code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

            with open('token.pkl', 'wb') as token:
                pickle.dump(creds, token)

    return creds
authenticate_google()

In [48]:
# Load credentials and initialize calendar_service
import pickle
from googleapiclient.discovery import build

with open('token.pkl', 'rb') as token:
    creds = pickle.load(token)

calendar_service = build('calendar', 'v3', credentials=creds)


## Agent Creation

In [49]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Gemini 2.0 Flash model setup (via LangChain)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3
)


The small_talk_chain is to ensure that the agent doesnt sound too formal. Having a friendly human touch at the front makes most users comfortable.

In [50]:
from langchain.prompts import PromptTemplate

small_talk_prompt = PromptTemplate.from_template("""
You are a polite AI assistant. The user said:

"{user_input}"

If this is a greeting, compliment, or general comment, respond warmly.
If it's a calendar-related request, say: "Let me check your calendar..."
If you're unsure, say: "Could you let me know if this is about your schedule?"

Keep it friendly, short, and helpful.
""")

small_talk_chain = small_talk_prompt | llm


The intent_chain is to detect the intent of the user. Is it calendar related? So that query can be appropriately routed.

In [51]:
intent_prompt = PromptTemplate.from_template("""
You are a calendar assistant. A user sent the following message:

"{user_input}"

Is the user trying to create, view, update, rename, reschedule, or delete calendar events?

Respond with only "yes" or "no".
""")

intent_chain = intent_prompt | llm


The planner chain converts the user input related to calendar into a valid json object

In [52]:
from langchain.prompts import PromptTemplate
from datetime import timedelta

# Date setup
today_str = datetime.datetime.now().strftime("%Y-%m-%d")
tomorrow_str = (datetime.datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

# Planner prompt with NO .format(), NO {curly braces} interpretation
prompt_text = f"""
Below are examples of calendar commands and how they should be converted into JSON:

Example 1:
User: What's on my calendar today?
Output:
{{{{"action_type": "list_events"}}}}

Example 2:
User: Schedule a sync with the team tomorrow at 2pm for 1 hour
Output:
{{{{"action_type": "create_event", "summary": "Team Sync", "start_time": "2025-04-19T14:00:00", "end_time": "2025-04-19T15:00:00", "timezone": "Asia/Kolkata"}}}}

Now it’s your turn.

Your job is to convert the following user input into a valid JSON object that includes:

- "action_type": must be one of:
  - "create_event"
  - "list_events"
  - "delete_event"
  - "update_event"

If action_type is "create_event", also include:
- summary
- start_time
- end_time
- timezone (use "Asia/Kolkata" by default)

⚠️ Respond with raw JSON only — no markdown, no triple backticks.

Today's date: {today_str}
Tomorrow's date: {tomorrow_str}

User: {{user_input}}

Output JSON:
"""

# Explicitly tell PromptTemplate we only want to substitute 'user_input'
planner_prompt = PromptTemplate(input_variables=["user_input"], template=prompt_text)
planner_chain = planner_prompt | llm


The Scheduler_chain is for carrying out operations in the google calendar using helper functions and tools

In [53]:
from langchain.prompts import PromptTemplate

# Scheduler Prompt Template
scheduler_prompt = PromptTemplate.from_template("""
You are a calendar scheduling assistant.

You will receive structured JSON input describing an event with fields:
- summary
- start_time
- end_time
- timezone

Your task:
- Validate the input
- Confirm if time/date seem valid (no obvious errors like 2AM meetings)
- If all looks good, return a message like: "Confirmed: scheduling event as described."
- If you detect issues, return a warning like: "Warning: time is unusual. Suggest revising."

Respond ONLY with a user-friendly confirmation or warning.

Here is the event:
{event_json}
""")


In [54]:
# LLM chain for scheduling logic
scheduler_chain = scheduler_prompt | llm

In [55]:
from langchain_core.runnables import RunnableLambda
import json



# Final scheduler agent
def scheduler_agent(event_data: dict):
    try:
        event_json = json.dumps(event_data, indent=2)
        confirmation = scheduler_chain.invoke({"event_json": event_json})
        print("Scheduler LLM Response:", confirmation)

        
        return create_event(
            summary=event_data["summary"],
            start_time=event_data["start_time"],
            end_time=event_data["end_time"],
            timezone=event_data.get("timezone", "Asia/Kolkata")
        )

    except Exception as e:
        return f" Scheduler failed: {e}"


## Helper functions and tools

In [56]:
# Helper: Get upcoming calendar events
def get_upcoming_events(n=5):
    import datetime

    now = datetime.datetime.utcnow().isoformat() + 'Z'
    events_result = calendar_service.events().list(
        calendarId='primary',
        timeMin=now,
        maxResults=n,
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    events = events_result.get('items', [])
    if not events:
        return "No upcoming events found."

    return "\n".join(
        f"{event['start'].get('dateTime', event['start'].get('date'))} - {event.get('summary', 'No Title')}"
        for event in events
    )

# Helper: Create calendar event
def create_event(summary: str, start_time: str, end_time: str, timezone: str = "UTC"):
    event = {
        'summary': summary,
        'start': {'dateTime': start_time, 'timeZone': timezone},
        'end': {'dateTime': end_time, 'timeZone': timezone},
    }
    created_event = calendar_service.events().insert(calendarId='primary', body=event).execute()
    return f" Event created: {created_event.get('htmlLink')}"


In [57]:
# Rename event by exact current title
def rename_event(current_title: str, new_title: str):
    events = calendar_service.events().list(
        calendarId='primary',
        q=current_title,
        singleEvents=True,
        orderBy='startTime'
    ).execute().get('items', [])

    if not events:
        return f" No event found with the title '{current_title}'"

    # Assume first match
    event = events[0]
    event['summary'] = new_title
    updated_event = calendar_service.events().update(
        calendarId='primary',
        eventId=event['id'],
        body=event
    ).execute()

    return f" Renamed event to: {updated_event['summary']}"

def delete_event(summary: str):
    print(f" Searching for event titled: '{summary}'")
    now = datetime.datetime.utcnow().isoformat() + "Z"

    events_result = calendar_service.events().list(
        calendarId="primary",
        timeMin=now,
        maxResults=50,
        singleEvents=True,
        orderBy="startTime"
    ).execute()
    events = events_result.get("items", [])

    deleted = []
    for event in events:
        event_summary = event.get("summary", "")
        if event_summary.lower() == summary.lower():
            calendar_service.events().delete(
                calendarId="primary", eventId=event["id"]
            ).execute()
            deleted.append(event_summary)

    if deleted:
        return f" Deleted {len(deleted)} event(s) titled: '{summary}'"
    else:
        return f" No event found with title matching '{summary}'"





In [58]:
#  Get upcoming events (wrapped)
def get_events_tool(_):
    return get_upcoming_events(n=5)

def create_event_tool(user_input):
    # Step 1: Gemini extracts structured intent
    planning_response = planner_chain.invoke({"user_input": user_input})


    import json, re
    cleaned_json = re.sub(r"```json|```", "", planning_response).strip()
    parsed = json.loads(cleaned_json)

    # Step 2: Use extracted values to create calendar event
    return create_event(
        summary=parsed["summary"],
        start_time=parsed["start_time"],
        end_time=parsed["end_time"],
        timezone=parsed.get("timezone", "Asia/Kolkata")
    )


In [59]:
from langchain.tools import Tool

calendar_tools = [
    Tool(
        name="GetCalendarEvents",
        func=get_events_tool,
        description="Fetches the next 5 calendar events."
    ),
    Tool(
        name="CreateCalendarEvent",
        func=create_event_tool,
        description="Create a new event from natural language."
    ),
    Tool(
        name="RenameCalendarEvent",
        func=lambda user_input: rename_event(*user_input.split(" to ")),
        description="Rename an existing event. Use format: 'current title to new title'"
    ),
    Tool(
    name="DeleteCalendarEvent",
    func=delete_event,
    description="Delete one or more calendar events by fuzzy title match (case-insensitive)."
    )

]


###  `multi_agent_calendar_handler(user_input)`

This function acts as the central **multi-agent controller** for your AI Calendar Assistant. It interprets user input, decides what action to take, and routes the request to the appropriate agent or tool.

---

####  What it does:

1. **Intent Detection**
   - Uses an LLM (`intent_chain`) to check whether the input is calendar-related.
   - If not, it replies with a friendly response using a small talk agent (`small_talk_chain`).

2. **Planning Phase**
   - Sends the input to the `planner_chain` (Gemini LLM) to generate a structured plan in JSON format.
   - Extracts and parses the planner's output.

3. **Action Type Routing**
   - Based on the parsed JSON, it checks the `action_type` field.
   - Supports:
     - `create_event` → handled by `scheduler_agent`
     - `list_events` → lists upcoming events
     - `delete_event` → deletes an event
     - `update_event` → (coming soon)

4. **Fallbacks**
   - Handles unknown actions or JSON decode errors gracefully.

---

####  Flow Diagram

```mermaid
graph TD
    A[User Input] --> B[Intent Detector]
    B -- Not Calendar Related --> C[Small Talk Agent]
    B -- Calendar Related --> D[Planner Agent - Gemini]
    D --> E{Action Type?}
    E -- create_event --> F[Scheduler Agent]
    E -- list_events --> G[List Events Tool]
    E -- delete_event --> H[Delete Tool]
    E -- update_event --> I[Coming Soon]
    E -- unknown --> J[Fallback Response]
```

---


In [60]:
def multi_agent_calendar_handler(user_input: str):
    # Step 0: Intent classification (calendar-related or not)
    intent = intent_chain.invoke({"user_input": user_input})
    if "no" in intent.content.lower():
        # Handle small talk
        reply = small_talk_chain.invoke({"user_input": user_input})
        return reply.content.strip() if hasattr(reply, "content") else str(reply)

    # Step 1: Planner LLM
    print("Planner Agent is analyzing...")
    planning_response = planner_chain.invoke({"user_input": user_input})

    # Extract content from Gemini LLM
    raw_output = planning_response.content if hasattr(planning_response, "content") else str(planning_response)
    print("Raw planner output:\n", raw_output)

    # Clean up JSON
    cleaned_json = re.sub(r"```json|```", "", raw_output).strip()

    try:
        parsed = json.loads(cleaned_json)
    except Exception as e:
        return f"JSON decode error: {e}\nRaw response: {raw_output}"

    # Step 2: Extract and check action_type
    action_type = parsed.get("action_type")

    if not action_type:
        return (
            " I couldn’t determine what action to take.\n"
            "Try rephrasing your request (e.g., 'list my events', 'schedule a sync tomorrow 2pm')."
        )

    # Step 3: Route to appropriate tool
    if action_type == "create_event":
        return scheduler_agent(parsed)

    elif action_type == "list_events":
        return get_upcoming_events()

    elif action_type == "delete_event":
        return delete_event(parsed.get("summary", ""))  # optional future support

    elif action_type == "update_event":
        return " Update support coming soon!"

    else:
        return f" Action '{action_type}' is not recognized yet."


In [ ]:
print(" Hello! I'm your AI Calendar Assistant.")
print("You can ask me to schedule, list, rename, or cancel events.")
print("Type 'exit' anytime to end the conversation.")
print("-" * 50)

while True:
    user_input = input("Ask me anything: ")
    if user_input.lower() in ["exit", "quit"]:
        print(" Goodbye!")
        break
    try:
        response = multi_agent_calendar_handler(user_input)
        print("\n Response:")
        print(response)
        print("-" * 50)
    except Exception as e:
        print(" Error:", e)
        print("-" * 50)


 Hello! I'm your AI Calendar Assistant.
You can ask me to schedule, list, rename, or cancel events.
Type 'exit' anytime to end the conversation.
--------------------------------------------------


Ask me anything:  list my events today


Planner Agent is analyzing...
Raw planner output:
 {"action_type": "list_events"}

 Response:
2025-04-19T14:00:00+05:30 - Team Meeting
2025-04-19T19:00:00+05:30 - client meeting
2025-04-19T20:00:00+05:30 - supplier meeting
--------------------------------------------------


Ask me anything:  schedule a team sync metting tomorrow at 3pm


Planner Agent is analyzing...
Raw planner output:
 {"action_type": "create_event", "summary": "Team Sync Meeting", "start_time": "2025-04-20T15:00:00", "end_time": "2025-04-20T16:00:00", "timezone": "Asia/Kolkata"}
Scheduler LLM Response: content='Confirmed: scheduling event as described.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run-160ab963-fa54-4db3-9261-4911cf9f9c92-0' usage_metadata={'input_tokens': 218, 'output_tokens': 8, 'total_tokens': 226, 'input_token_details': {'cache_read': 0}}

 Response:
 Event created: https://www.google.com/calendar/event?eid=NW5xdjB0bGtwYzZzZjJkOGdmN2VhbThlMzggc2F0aGlzaGt1bWFydGhldGFAbQ
--------------------------------------------------
